In [ ]:
%pip install wandelbots
%pip install python-dotenv

In [1]:
import os
from dotenv import load_dotenv
from wandelbots import Instance, MotionGroup, Planner
from wandelbots.types import Pose

load_dotenv()

True

In [3]:
os.environ["WANDELAPI_BASE_URL"] = "https://instance.wandelbots.io"
os.environ["NOVA_USERNAME"] = "wb"
os.environ["NOVA_PASSWORD"] = "your_password"
os.environ["CELL_ID"] = "cell"
os.environ["MOTION_GROUP"] = "0@myrobot"
os.environ["TCP"] = "Flange"

## 🤖 Show Robot-Pad

In [2]:
from IPython.display import IFrame
IFrame(src=os.getenv("WANDELAPI_BASE_URL") + '/cell/robot-pad', width=700, height=600)

## ⚙️ Configure Robot

In [3]:
my_instance = Instance(
    url=os.getenv("WANDELAPI_BASE_URL"),
    user=os.getenv("NOVA_USERNAME"),
    password=os.getenv("NOVA_PASSWORD"),
)

my_robot = MotionGroup(
    instance=my_instance,
    cell=os.getenv("CELL_ID"),
    motion_group=os.getenv("MOTION_GROUP"),
    default_tcp=os.getenv("TCP"),
)

## 🔩 Get Robot Props & State

In [4]:
print(my_robot.tcps())
print([round(j, 2) for j in my_robot.current_joints()])
print(my_robot.current_tcp_pose())

['Flange']
[0.0, -1.57, -1.54, -1.6, 1.57, 0.0]
Position: [  691,  -174,   693]
Rotation: [ 2.22, -2.22, -0.00]


## 📝 Perform Pose Transforms

In [5]:
current_pose = my_robot.current_tcp_pose()
translation = Pose.from_list([0, 0, -400, 0, 0, 0])
target_pose = translation * current_pose
print(f"Current\n{current_pose}\n\nTarget\n{target_pose}")

Current
Position: [  691,  -174,   693]
Rotation: [ 2.22, -2.22, -0.00]

Target
Position: [  691,  -174,   293]
Rotation: [ 2.22, -2.22, -0.00]


## 🎢 Plan a Motion

In [6]:
HOME = my_robot.current_joints()

planner = Planner(my_robot)
trajectory = [
    planner.jptp(target=HOME),
    planner.set_io(key="tool_out[0]", value=True),
    planner.line(target=target_pose),
    planner.set_io(key="tool_out[1]", value=False),
    planner.jptp(target=HOME),
]
plan_result, io_actions = planner.plan(start_joints=my_robot.current_joints(), trajectory=trajectory)

print("motion id:", plan_result.motion)

motion id: 5dc62210-7012-4a68-a54d-1c962ce7179b


## 🏃🏽‍♀️ Execute Motion

In [7]:
io_names = [action.io.key for action in io_actions]
ios = my_robot.get_ios(io_names)
print(f"{ios[0]}\n{ios[1]}")

tool_out[0]: True
tool_out[1]: False


In [8]:
result = await my_robot.execute_async(motion=plan_result.motion, speed=25, io_actions=io_actions)

In [9]:
ios = my_robot.get_ios(io_names)
print(f"{ios[0]}\n{ios[1]}")

tool_out[0]: True
tool_out[1]: False
